In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
from scipy import stats
import matplotlib.pyplot as plt
import pickle
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# TRAIN ADS

In [2]:
df_train_ads = pd.read_csv('CTR Prediction - 2022 DIGIX Global AI Challenge/train/train_data_ads.csv')
df_train_ads.head()

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,373250,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
1,373253,1,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
2,373252,1,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
3,373251,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
4,373255,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030328,39^220^16,0,15


In [43]:
df_train_ads.shape

(7675517, 35)

# TRAIN FEEDS

In [46]:
df_train_feeds = pd.read_csv('CTR Prediction - 2022 DIGIX Global AI Challenge/train/train_data_feeds.csv')

In [47]:
df_train_feeds.shape

(3227732, 28)

# TEST ADS

In [48]:
df_test_ads = pd.read_csv('CTR Prediction - 2022 DIGIX Global AI Challenge/test/test_data_ads.csv')

In [49]:
df_test_ads.shape

(976058, 34)

# TEST FEEDS

In [50]:
df_test_feeds = pd.read_csv('CTR Prediction - 2022 DIGIX Global AI Challenge/test/test_data_feeds.csv')
df_test_feeds.head()

,u_userId,u_phonePrice,u_browserLifeCycle,u_browserMode,u_feedLifeCycle,u_refreshTimes,u_newsCatInterests,u_newsCatDislike,u_newsCatInterestsST,u_click_ca2_news,...,e_ch,e_m,e_po,e_pl,e_rn,e_section,e_et,label,cillabel,pro
0,215116,16,17,14,11,0,65^72^42^65^104,0,219^50^151^21^140,219^50^151^114^21,...,19,998,9,2449,13,0,202206100922,-1,-1,0
1,215116,16,17,14,11,0,65^72^42^65^104,0,219^50^151^21^140,219^50^151^114^21,...,19,998,7,2449,14,0,202206100924,-1,-1,0
2,215116,16,17,14,11,0,65^72^42^65^104,0,0,65^104^98^168^0,...,19,998,14,2449,1,0,202206100907,1,-1,40
3,215116,16,17,14,11,0,65^72^42^65^104,0,219^50^151^21^140,219^50^151^114^21,...,19,998,5,2449,11,0,202206100921,-1,-1,0
4,215116,16,17,14,11,0,65^72^42^65^104,0,219^50^151^21^140,219^50^151^114^21,...,19,998,3,2449,14,0,202206100922,-1,-1,0


In [51]:
df_test_feeds.shape

(369341, 28)

# FIND OUT THE TARGET

In [52]:
df_test_ads.columns

Index(['log_id', 'user_id', 'age', 'gender', 'residence', 'city', 'city_rank',
       'series_dev', 'series_group', 'emui_dev', 'device_name', 'device_size',
       'net_type', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
       'hispace_app_tags', 'app_second_class', 'app_score',
       'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
       'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003',
       'pt_d', 'u_newsCatInterestsST', 'u_refreshTimes', 'u_feedLifeCycle'],
      dtype='object')

In [53]:
df_train_ads.columns

Index(['log_id', 'label', 'user_id', 'age', 'gender', 'residence', 'city',
       'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name',
       'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd',
       'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
       'hispace_app_tags', 'app_second_class', 'app_score',
       'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
       'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003',
       'pt_d', 'u_newsCatInterestsST', 'u_refreshTimes', 'u_feedLifeCycle'],
      dtype='object')

In [54]:
df_train_ads.label.value_counts(normalize=True)

0    0.984478
1    0.015522
Name: label, dtype: float64

the label should be it

# CLEANING TIME!

there are no nan and duplicated in this dataset

In [3]:
df = df_train_ads.copy()

In [4]:
df.columns

Index(['log_id', 'label', 'user_id', 'age', 'gender', 'residence', 'city',
       'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name',
       'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd',
       'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
       'hispace_app_tags', 'app_second_class', 'app_score',
       'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
       'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003',
       'pt_d', 'u_newsCatInterestsST', 'u_refreshTimes', 'u_feedLifeCycle'],
      dtype='object')

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7675517 entries, 0 to 7675516
Data columns (total 35 columns):
 #   Column                Dtype  
---  ------                -----  
 0   log_id                int64  
 1   label                 int64  
 2   user_id               int64  
 3   age                   int64  
 4   gender                int64  
 5   residence             int64  
 6   city                  int64  
 7   city_rank             int64  
 8   series_dev            int64  
 9   series_group          int64  
 10  emui_dev              int64  
 11  device_name           int64  
 12  device_size           int64  
 13  net_type              int64  
 14  task_id               int64  
 15  adv_id                int64  
 16  creat_type_cd         int64  
 17  adv_prim_id           int64  
 18  inter_type_cd         int64  
 19  slot_id               int64  
 20  site_id               int64  
 21  spread_app_id         int64  
 22  hispace_app_tags      int64  
 23  app_sec

# pt_d
time

In [5]:
df.pt_d.head()

0    202206030326
1    202206030326
2    202206030326
3    202206030326
4    202206030328
Name: pt_d, dtype: int64

In [6]:
df.pt_d = pd.to_datetime(df.pt_d, format='%Y%m%d%H%M')

In [7]:
# check for number of uniques for each col
col = df.columns
for c in col:
    print(c, len(df[c].unique()))

log_id 1176633
label 2
user_id 65297
age 8
gender 3
residence 35
city 341
city_rank 4
series_dev 27
series_group 7
emui_dev 27
device_name 256
device_size 1547
net_type 6
task_id 11209
adv_id 12615
creat_type_cd 9
adv_prim_id 545
inter_type_cd 4
slot_id 60
site_id 1
spread_app_id 116
hispace_app_tags 43
app_second_class 20
app_score 3
ad_click_list_v001 108720
ad_click_list_v002 95376
ad_click_list_v003 61102
ad_close_list_v001 3883
ad_close_list_v002 2701
ad_close_list_v003 1715
pt_d 5436
u_newsCatInterestsST 187576
u_refreshTimes 10
u_feedLifeCycle 8


### ad_click_list & ad_close_list & u_newsCatInterests

they all have the format of:

1111^2222^3333

ad_click_list_001: 用户点击广告任务 id 列表 -> the id list of ad tasks that the user clicks

ad_close_list: 用户点击广告对应广告主 id 列表


In [8]:
df.ad_click_list_v001.sample()

3269059    30658
Name: ad_click_list_v001, dtype: object

In [10]:
col_sel = ['ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
       'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003', 
       'u_newsCatInterestsST']

In [11]:
for col in col_sel:
    df[col] = df[col].map(lambda x: x.split('^'))

In [12]:
df.ad_click_list_v003.sample()

4530943    [246, 312, 304, 131, 321]
Name: ad_click_list_v003, dtype: object

# save to CSV, and call it a day!

### baseline dataframe select

In [143]:
col_sel = ['log_id','user_id','site_id','ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
       'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003','pt_d', 'u_newsCatInterestsST']
df_export = df.drop(columns=col_sel)

In [144]:
df_export.columns

Index(['label', 'age', 'gender', 'residence', 'city', 'city_rank',
       'series_dev', 'series_group', 'emui_dev', 'device_name', 'device_size',
       'net_type', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'inter_type_cd', 'slot_id', 'spread_app_id', 'hispace_app_tags',
       'app_second_class', 'app_score', 'u_refreshTimes', 'u_feedLifeCycle'],
      dtype='object')

In [146]:
df_export.to_csv(r'C:\Users\qwswe\Documents\Flatiron\DS_P3\Output_Data\Model_1_baseline_data.csv', index = False)

### dataframe for eda

In [13]:
df.columns

Index(['log_id', 'label', 'user_id', 'age', 'gender', 'residence', 'city',
       'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name',
       'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd',
       'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
       'hispace_app_tags', 'app_second_class', 'app_score',
       'ad_click_list_v001', 'ad_click_list_v002', 'ad_click_list_v003',
       'ad_close_list_v001', 'ad_close_list_v002', 'ad_close_list_v003',
       'pt_d', 'u_newsCatInterestsST', 'u_refreshTimes', 'u_feedLifeCycle'],
      dtype='object')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7675517 entries, 0 to 7675516
Data columns (total 35 columns):
 #   Column                Dtype         
---  ------                -----         
 0   log_id                int64         
 1   label                 int64         
 2   user_id               int64         
 3   age                   int64         
 4   gender                int64         
 5   residence             int64         
 6   city                  int64         
 7   city_rank             int64         
 8   series_dev            int64         
 9   series_group          int64         
 10  emui_dev              int64         
 11  device_name           int64         
 12  device_size           int64         
 13  net_type              int64         
 14  task_id               int64         
 15  adv_id                int64         
 16  creat_type_cd         int64         
 17  adv_prim_id           int64         
 18  inter_type_cd         int64         
 19  

In [17]:
df.to_csv(r'D:\VIVY_LEARNING\Flatiron\DS_P3_CTR_Prediction\Output_Data\EDA_data.csv', index = False)